In [1]:
!pip install datatune

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.4/278.4 kB 18.0 MB/s eta 0:00:00


Use the following command to download the customers dataset. You can also download the dataset from here: https://www.datablist.com/learn/csv/download-sample-csv-files#download-customers-sample-csv-files

In [2]:
!gdown --id 1zO8ekHWx9U7mrbx_0Hoxxu6od7uxJqWw


/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1zO8ekHWx9U7mrbx_0Hoxxu6od7uxJqWw
To: /content/customers-100.csv
100% 17.3k/17.3k [00:00<00:00, 30.8MB/s]


In [3]:
import os
import dask.dataframe as dd
import datatune as dt
from datatune.llm.llm import Azure

In [4]:
# set your LLM API keys.(In this tutorial, we will use Azure) Alternatively, if you're using your custom LLM, check out https://docs.datatune.ai/LLM.html

os.environ["AZURE_API_KEY"] = "Your Azure API KEY"
os.environ["AZURE_API_BASE"] = "Your Azure API Base"
os.environ["AZURE_API_VERSION"] = "Your API Version"


In [5]:
llm = Azure(
    model_name="gpt-4o-mini",
    api_key=os.getenv("AZURE_API_KEY"),
    api_base=os.getenv("AZURE_API_BASE"),
    api_version=os.getenv("AZURE_API_VERSION"),
    tpm=1000000,
    rpm=1000
)

In [6]:
# Create a Datatune Agent
agent = dt.Agent(llm)

In [7]:
# load data as a dask dataframe
df = dd.read_csv('/content/customers-100.csv')

In [8]:
# explore the data
df.head(10)

,Index,Customer Id,First Name,Last Name,Company,City,Country,Phone 1,Phone 2,Email,Subscription Date,Website
0,1,DD37Cf93aecA6Dc,Sheryl,Baxter,Rasmussen Group,East Leonard,Chile,229.077.5154,397.884.0519x718,zunigavanessa@smith.info,2020-08-24,http://www.stephenson.com/
1,2,1Ef7b82A4CAAD10,Preston,Lozano,Vega-Gentry,East Jimmychester,Djibouti,5153435776,686-620-1820x944,vmata@colon.com,2021-04-23,http://www.hobbs.com/
2,3,6F94879bDAfE5a6,Roy,Berry,Murillo-Perry,Isabelborough,Antigua and Barbuda,+1-539-402-0259,(496)978-3969x58947,beckycarr@hogan.com,2020-03-25,http://www.lawrence.com/
3,4,5Cef8BFA16c5e3c,Linda,Olsen,"Dominguez, Mcmillan and Donovan",Bensonview,Dominican Republic,001-808-617-6467x12895,+1-813-324-8756,stanleyblackwell@benson.org,2020-06-02,http://www.good-lyons.com/
4,5,053d585Ab6b3159,Joanna,Bender,"Martin, Lang and Andrade",West Priscilla,Slovakia (Slovak Republic),001-234-203-0635x76146,001-199-446-3860x3486,colinalvarado@miles.net,2021-04-17,https://goodwin-ingram.com/
5,6,2d08FB17EE273F4,Aimee,Downs,Steele Group,Chavezborough,Bosnia and Herzegovina,(283)437-3886x88321,999-728-1637,louis27@gilbert.com,2020-02-25,http://www.berger.net/
6,7,EA4d384DfDbBf77,Darren,Peck,"Lester, Woodard and Mitchell",Lake Ana,Pitcairn Islands,(496)452-6181x3291,+1-247-266-0963x4995,tgates@cantrell.com,2021-08-24,https://www.le.com/
7,8,0e04AFde9f225dE,Brett,Mullen,"Sanford, Davenport and Giles",Kimport,Bulgaria,001-583-352-7197x297,001-333-145-0369,asnow@colon.com,2021-04-12,https://hammond-ramsey.com/
8,9,C2dE4dEEc489ae0,Sheryl,Meyers,Browning-Simon,Robersonstad,Cyprus,854-138-4911x5772,+1-448-910-2276x729,mariokhan@ryan-pope.org,2020-01-13,https://www.bullock.net/
9,10,8C2811a503C7c5a,Michelle,Gallagher,Beck-Hendrix,Elaineberg,Timor-Leste,739.218.2516x459,001-054-401-0347x617,mdyer@escobar.net,2021-11-08,https://arias.com/


In [12]:
# Create a sample anonymisation prompt
prompt = """
"1)Filter only for Asian countries.
 2)Personally identifiable information for women in data should be replaced with 'XX' - emails, phone numbers, names, addresses""
"""

In [13]:
df_anon = agent.do(prompt, df)
df_anon.to_csv("anonymised_out.csv", index=False)
print("Enhanced anonymised dataset saved to anonymised_out.csv")

Enhanced anonymised dataset saved to anonymised_out.csv


In [14]:
print(df_anon.head(10))

    Index      Customer Id First Name Last Name                     Company  \
8       9  C2dE4dEEc489ae0         XX        XX              Browning-Simon   
9      10  8C2811a503C7c5a         XX        XX                Beck-Hendrix   
11     12  CEDec94deE6d69B         XX        XX   Hoffman, Reed and Mcclain   
13     14  A08A8aF8BE9FaD4         XX        XX              Carpenter-Cook   
14     15  6fEaA1b7cab7B6C         XX        XX              Carter-Hancock   
15     16  8cad0b4CBceaeec         XX        XX          Singleton and Sons   
23     24  010468dAA11382c         XX        XX             Watts-Donaldson   
45     46  fD780ED8dbEae7B         XX        XX  Price, Sexton and Mcdaniel   
46     47  300A40d3ce24bBA   Geoffrey    Guzman               Short-Wiggins   
48     49  F4Fc91fEAEad286      Brady     Cohen            Osborne-Erickson   

                 City                           Country           Phone 1  \
8                  XX                            Cypr